# Ramda - 你的函數化開發工具箱

## META

- LOG: 2017-08-09: created, 2017-08-31: refine shell tool, 2019-01-10: port to learn-js-in-jupyter

### 前置作業 - 讀取 shell 指令工具

In [ ]:
// prettier-ignore
var shellCmd = (cmdStr, silent=false, waitMsg='💡 執行中...不要走開...', successMsg='✅ 執行成功', failMsg='⚠️ 指令失敗') => {var cmdProcess = require('child_process').exec(cmdStr); silent ? null : (console.log(waitMsg) || cmdProcess.stdout.on('data', data => console.log(data))); cmdProcess.on('close', code => code !== 0 ? console.log(`${failMsg}: "${cmdStr}" error code: ${code}`) : console.log(successMsg))}
// prettier-ignore
var log = (...args) => {console.log(...args); var BEEP = true; (BEEP || true) ? process.stdout.write('\x07') : null};
log('✅ 執行成功');

### 前置作業 - 安裝 ramda 模組

In [ ]:
shellCmd('npm install ramda');

#### 已經安裝過了？檢查你的 npm 套件清單

In [ ]:
shellCmd('npm list --depth=0 2>/dev/null');

### 前置作業 - 讀取 ramda 模組

In [ ]:
const R = require('ramda');

---

#### Review of [Thinking in Ramda](http://randycoulman.com/blog/categories/thinking-in-ramda/)

#### Notes

- JS function explaination: __Reusable piece of code__
  - with zero or more inputs and outputs

- this article will be _less academic_ to welcome more readers
- Use ramda for example allow you do fp in JS in a _clean and elegant_ way

- for js function being __pure__ is what we hope
  - if pure then we can _derive_ expression like what we do in math class
    - pure function in JS is function in math class: same input always result same output


---

### I. Starts from Array prototype function

- almost every array prototype function is available in Ramda
- http://randycoulman.com/blog/2016/05/24/thinking-in-ramda-getting-started/

---

#### `forEach()`: for loop in FP way

In [ ]:
{
  const scores = [50, 80, 90, 100];

  R.forEach(value => console.log(value), scores); // return array
  //   scores.forEach(value => console.log(value));  // JS built-in: returns undefined
}

---

#### `map()`: create new array with value transformation

In [ ]:
{
  const prices = [50, 80, 90, 100];

  const discount = x => x * 0.8;
  // function can be a variable,

  // so function can be argument!
  R.map(discount, prices);

  // prices.map(discount);  // JS built-in
}

---

#### `filter()`, `reject()`: create new array to remove unwanted values

In [ ]:
{
  const scores = [50, 80, 90, 100];

  const passTest = x => x > 60; // number => boolean

  R.filter(passTest, scores);

  // scores.filter(passTest);  // JS built-in
}

In [ ]:
{
  const isEven = x => x % 2 === 0;
  R.reject(isEven, [1, 2, 3, 4, 5]);
}

---

### II. Combining functions

- 💡 pass functions to other functions
- http://randycoulman.com/blog/2016/05/31/thinking-in-ramda-combining-functions/

#### `complement()`: create new function where calculates __"NOT" value__ of original output

In [ ]:
{
  const isEven = x => x % 2 === 0;
  const isOdd = R.complement(isEven);

  R.find(isOdd, [1, 2, 3, 4, 5]);
}

---

#### `both()`, `either()`: create new function where calculates __"AND" value__, __"OR" value__ of original output

In [ ]:
{
  const customers = [
    { age: 20, isMarried: true, gender: 'MALE' },
    { age: 29, isMarried: false, gender: 'MALE' },
    { age: 15, isMarried: false, gender: 'FEMALE' },
  ];
  const isTeenager = c => c.age <= 18;
  const isFemale = c => c.gender === 'FEMALE';
  const isMarried = c => c.isMarried === true;

  const isFemaleTeenager = R.both(isFemale, isTeenager); // R.allPass([isFemale, isTeenager])
  const isMarriedOrFemale = R.either(isMarried, isFemale); // R.anyPass([isMarried, isFemale])

  log(customers.filter(isFemaleTeenager));
  log(customers.filter(isMarriedOrFemale));
}

---

#### `pipe()`, `compose()`: run functions in sequence as one pipeline

- 💡 Ramda 既然是 functional programming 工具箱，將零件 -- 函式 -- 組合成新的函式，絕對是常用且重要的功能
- [`pipe`](pipe) 的行為是 **由左到右組合函式**，[`compose`][compose] 的行為相反，**是從右到左組合函式**
- 換句話說，pipe, compose 都是 Higher order function（組合函式的函式，它的產出也是一個函式）

[pipe]: http://ramdajs.com/docs/#pipe "pipe"
[compose]: http://ramdajs.com/docs/#compose "compose"

In [ ]:
{
  const multiplyTen = x => x * 10;
  let adjustScore = R.pipe(
    Math.sqrt,
    multiplyTen,
    Math.floor
  );

  log(adjustScore(80));

  adjustScore = R.compose(
    Math.floor,
    multiplyTen,
    Math.sqrt
  );

  log(adjustScore(88));
}

- 什麼時侯用 pipe? 什麼時候用 compose? 
  - 看情況，閱讀順序如果是左到右，就用 pipe，但是如果想要表現 nested 函式（`f(g(x))`），就用 compose

---

### III. Partial Function

- Reason: function itself can be passed around other functions
  - sometimes we need _only part of it's arguments being set, not called too early_
- `R.partial` creates new func with only partial args being set
- `R.curry` creates new func that you can set args partially in `f(a)(b)` or `f(a,b)` style
- http://randycoulman.com/blog/2016/06/07/thinking-in-ramda-partial-application/

---

#### `partial()`, `curry()`: create new function with _partial arguments_ set

In [ ]:
{
  // 1. multiple args + R.partial
  const scores = [50, 80, 90, 100];

  let passTest = function(score, x) {
    return x > score;
  };
  R.filter(R.partial(passTest, [80]), scores);
}

In [ ]:
{
  // 2. Higher order function
  const scores = [50, 80, 90, 100];

  let passTest = function(score) {
    return x => x > score;
  };
  passTest = score => x => x > score;
  R.filter(passTest(80), scores);
}

In [ ]:
{
  // 3. R.curry
  const scores = [50, 80, 90, 100];

  let passTest = function(score, x) {
    return x > score;
  };
  passTest = R.curry(passTest); // curry a multiple args function

  R.filter(passTest(90), scores);
  /**
   * NOTE using curried func needs 2 steps:
   * 1. config (passing value to `score` arg)
   * 2. call it (`passTest(90)` is a function applied to scores array)
   */
}

---

#### `flip()`: create new function with _arguments in reversed order_

In [ ]:
{
  const scores = [50, 80, 90, 100];

  const passTest = function(x, score) {
    return x > score;
  };
  // NOTE if argument order is fixed, we use flip to reverse args
  R.filter(R.curry(R.flip(passTest))(70), scores);
}

---

#### `__`: argument placeholder for curried function

- `curry` + `__` : freely pre-assign argument(s) before a function is called
- ⚠️ `__` works only for curried functions
  - c.f. `partial`, `partialRight`, `flip` works on any function

In [ ]:
{
  const distanceToOrigin3D = R.curry((x, y, z) =>
    Math.sqrt(x * x + y * y, +z * z)
  );
  const distanceToOrigin2D = distanceToOrigin3D(R.__, R.__, 0);
  const distanceToOrigin1D = distanceToOrigin3D(R.__, 0, 0);

  log(distanceToOrigin2D(3, 4));
  log(distanceToOrigin1D(-20));
}

---

#### 💡 Almost every Ramda function is _Curried_

- `R.filter(passTest, scores)` is equilavent to `R.filter(passTest)(scores)`
- `R.filter(g, R.map(f, scores))` can be revied
  - ➡ `R.filter(g)(R.map(f, scores))`
  - ➡ `R.filter(g)(R.map(f)(scores))`
  - ➡ `R.pipe(R.filter(g), R.map(f))(scores)`

In [ ]:
{
  // revisit 2. Higher order function with all Ramda tools
  const scores = [50, 80, 90, 100];

  const multiplyTen = x => x * 10;
  const adjustScore = R.pipe(
    Math.sqrt,
    multiplyTen,
    Math.floor
  );

  const passTest = function(x, score) {
    return x > score;
  };
  const passTestCurried = R.curry((x, score) => x > score);
  const passSixty = passTestCurried(R.__, 60);

  R.pipe(
    R.map(adjustScore),
    R.filter(passSixty)
  )(scores);
}

---

### Imperative vs Declarative

- _telling how to do it_ vs _telling what we want_
- express tasks in a declarative way

---

#### `multiply()`, `add()`, `inc()`: Arithmetic operation in Ramda

- Ramda also provides `substract()`, `divide()`, `dec()`, `negate()`

In [ ]:
{
  const square = x => R.multiply(x, x);

  const operate = R.pipe(
    R.multiply,
    R.add(1), // R.inc
    square
  );

  operate(2, 3);
}

---

#### `equals()`, `gt()`, `lt()`: Relation operation in Ramda

- Ramda also provides `gte()`, `lte()`, `identical()`
- ⚠️ `gt()`, `gte()`, `lt()`, `lte()` is NOT "config first, data last" style
  - `gt(a, b)` means "_is a greater than b?_"
  - we may leverage placeholder to enhance it

In [ ]:
{
  const scores = [50, 80, 90, 100];

  const passTest = R.gt(R.__, 60); // R.gt is already curried!
  const notPerfect = R.flip(R.lt)(100); // Or using R.flip()

  R.pipe(
    R.filter(passTest),
    R.filter(notPerfect)
  )(scores);
}

---

#### `and()`, `or()`, `not()`, `isEmpty()`, `isNil()`: Common logic or type checks of values

- 💡 `and()`, `or()`, `not()` works __on values__
  - c.f. `both()`, `either()`, `complement()` works __on functions__
- [`isEmpty()`][isEmpty] checks value is its empty value
  - `[]` for array
  - `''` for string
  - `{}` for object
- [`isNil()`][isNil] checks if value is `null` or `undefined`


[isEmpty]: https://ramdajs.com/docs/#isEmpty
[isNil]: https://ramdajs.com/docs/#isNil

In [ ]:
{
  R.isEmpty({ length: 0 }); // => false; Safer than x => x.length === 0
}

---

#### [`defaultTo()`][defaultTo]: Safe and declarative to way to set default value

- Returns 2nd argument if it is __NOT__ `null`, `undefined` or `NaN`; otherwise 1st argument is returned.

[defaultTo]: https://ramdajs.com/docs/#defaultTo

In [ ]:
{
  const defaultTo42 = R.defaultTo(42);

  log(defaultTo42(null)); //=> 42
  log(defaultTo42(undefined)); //=> 42
  log(defaultTo42(false)); //=> false
  log(defaultTo42('Ramda')); //=> 'Ramda'

  // parseInt('string') results in NaN
  log(defaultTo42(parseInt('string'))); //=> 42
}

### `ifElse()`, `always()`, `identity()`: Conditional, Constants, Identity

- [`identity()`][identity] returns always __first argument__
  - if want to return other argument use [`nthArg()`][nthArg]
  
[identity]:  https://ramdajs.com/docs/#identity
[nthArg]:  https://ramdajs.com/docs/#nthArg

In [ ]:
{
  const alwaysPassTest = score =>
    R.ifElse(R.lt(R.__, 60), R.always(60), R.identity)(score);

  log(alwaysPassTest(100));
  log(alwaysPassTest(88));
  log(alwaysPassTest(60));
  log(alwaysPassTest(59));
}

---

### `when()`, `unless()`: Shorthand for ifElse + identity

In [ ]:
{
  let alwaysPassTest = score => R.when(R.lt(R.__, 60), R.always(60))(score);

  log(alwaysPassTest(100));
  log(alwaysPassTest(88));
  log(alwaysPassTest(60));
  log(alwaysPassTest(59));

  alwaysPassTest = score => R.unless(R.gte(R.__, 60), R.always(60))(score);

  log(alwaysPassTest(100));
  log(alwaysPassTest(88));
  log(alwaysPassTest(60));
  log(alwaysPassTest(59));
}

---

### `cond()`: switch statement in functional programming

In [ ]:
{
  const water = temperature =>
    R.cond([
      [R.equals(0), R.always('water freezes at 0°C')],
      [R.equals(100), R.always('water boils at 100°C')],
      [R.T, temp => `nothing special happens at ${temp}°C`],
    ])(temperature);

  log(water(50));
  log(water(0));
  log(water(100));
}

---

## __WIP__

---

## FAQ

#### Q1 ❓Ramda 工具箱裡有多少工具可以用？

In [ ]:
log(`💡Ramda 有 ${Object.keys(R).length} 個函式可以用`);

#### Q2 ❓Ramda 的 API 這麼多，有分類嗎？

> Ramda 官方文件將 Ramda 的 API 加上標籤分類，包含了:

- `Math` (計算)
- `Logic` (條件判斷)
- `Type` (變數類型判斷)
- `Relation` (變數之間的比較)
- `List` (陣列操作)
- `Object` (物件操作)
- `Function` (高階函式)

_NOTE_ magic words e.g. `% env`, `% bash` is not working in `jupyter-nodejs`
Those examples are not working:

```jupyter

%%bash
pwd
```

```jupyter

%%HTML
<a href="https://www.google.com">Google Link</a>
```

## === TODO top 20 to intro

pipe
always
prop
path
equals
assoc
isNil
find
ifElse
cond
contains
identity
pathOr
map
T
head
isEmpty
pick
assocPath
compose

### [邏輯] 用 ifElse 執行判斷

### 用 pipe 把函式的流程串起來

### [物件相關] path, pick, omit, over

### [通用] nthArg() 抓第幾個參數

In [ ]:
/*
empty()
a -> a

Returns the empty value of its argument's type.
Ramda defines the empty value of
Array ([]), Object ({}), String (''), and Arguments.

Other types are supported if they define <Type>.empty and/or <Type>.prototype.empty.

Dispatches to the empty method of the first argument, if present.
*/

// console.log(R.empty(Just(42)))
//=> Nothing()
console.log(R.empty([1, 2, 3]));
//=> []
console.log(R.empty('unicorns'));
//=> ''
console.log(R.empty({ x: 1, y: 2 }));
//=> {}
console.log(R.empty(undefined));
// => undefined
console.log(R.empty(null));
// => undefined

In [ ]:
/*
isEmpty()
a -> boolean

Returns true if the given value is its type's empty value; false otherwise.

[], {}, ''
*/

console.log('testing isEmpty()\n');

console.log(R.isEmpty([1, 2, 3]));
//=> false
console.log(R.isEmpty([]));
//=> true
console.log(R.isEmpty(''));
//=> true
console.log(R.isEmpty(null));
//=> false
console.log(R.isEmpty({}));
//=> true
console.log(R.isEmpty({ length: 0 }));
//=> false
console.log(R.isEmpty(undefined));
// => false NOTE not returning result

In [ ]:
// test some api of ramda

const demoCurry = (() => {
  const addFourNumbers = (a, b, c, d) => a + b + c + d;

  const curriedAddFourNumbers = R.curry(addFourNumbers);
  const f = curriedAddFourNumbers(1, 2);
  const g = f(3);
  const res = g(4); //=> 10

  console.log(res);
  // console.log(res());  // NOTE error

  const curriedMultiplyFourNumbers = R.curry((w, x, y, z) => w * x * y * z);
  console.log(curriedMultiplyFourNumbers(1, 2)(4)(8)); // 64
})();

const demoPipe_Match = (() => {
  // left-to-right compose
  const fn = R.pipe(
    R.match(/\/static/g),
    R.isEmpty
  );
  console.log(fn(''));

  // match is a curried function
  console.log(String(R.match(/\/static/g)));
  console.log(String(R.match));

  console.log(R.match(/\/foo/)('/foo'));
  console.log(R.match()('foo')('foooo'));
})();

const demoProp_isEmpty = (() => {
  // replace prop
  console.log(R.prop()('xx')()({ xx: 100 }));
  console.log(R.prop('xx', { xx: 100 }));
  console.log(R.isEmpty()()()(''));
})();